In [2]:
# IMPORTS
import os, shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Supress tensorflow log outputs

In [3]:
# DATA PREPROCESSING
# Original dataset directory
path = "/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset"

# Create fish dataset directory
!mkdir fish_dataset

# Iterate through all folders in directory and creates list of fishes
fishes = []
for name in os.listdir(path):
    if os.path.isdir(os.path.join(path, name)):
        fishes.append(name)
        
# Copy over relavant folders to fish dataset directory (takes a while)
for fish in fishes:
    fish_image_folder = os.path.join(path, fish, fish)
    shutil.copytree(fish_image_folder, os.path.join("./fish_dataset", fish))

In [4]:
"""
Directory should now have the following structure so we can use tf.keras.preprocessing.image_dataset_from_directory:
main_directory/
...class_a/
......a_image_1.jpg
......a_image_2.jpg
...class_b/
......b_image_1.jpg
......b_image_2.jpg
"""
print(os.listdir("./fish_dataset"))
print(os.listdir("./fish_dataset/Gilt-Head Bream")[:3])

In [5]:
# CREATING TRAINING AND VALIDATION DATASET
img_size = (224, 224)
batch_size = 32

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    "./fish_dataset/",
    labels="inferred",
    label_mode="categorical",  
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,  
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="training",
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    "./fish_dataset/",
    labels="inferred",
    label_mode="categorical",  
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,  
    shuffle=True,
    seed=1,
    validation_split=0.2,
    subset="validation",
)

In [6]:
# CNN MODEL BUILDING
model = keras.Sequential([
    layers.Input((224, 224, 3)),
    
    layers.Conv2D(16, 3, activation="relu"),
    layers.BatchNormalization(), # Regularization
    layers.MaxPooling2D(), 
    
    layers.Conv2D(32, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2), # Regularization
    layers.Dense(9, activation="softmax"),
])

In [7]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    loss = keras.losses.CategoricalCrossentropy(),
    metrics = ["accuracy"],
)

In [15]:
model.summary()

In [8]:
model.fit(ds_train, batch_size=64, epochs=5, verbose=2, validation_data=ds_validation)

In [13]:
ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    "./fish_dataset/",
    labels="inferred",
    label_mode="categorical",  
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,  
    shuffle=True,
    seed=1,
    validation_split=0.6,
    subset="validation",
)

In [14]:
model.evaluate(ds_test)

### ACCURACY OF THE MODEL IS 99.96%